# A guide through compilation in Python using Electrostatics

On the computation of the follow values

$$\sum_{i >j } \frac{q_i q_j}{r_{ij}}$$

Install any packages not available.  
`%load_ext cython` is an example of cell magic. This one enables auto-compilation with Cython.

In [ ]:
import numpy as np
import cython
%load_ext cython
%matplotlib inline

Create a sytem of particles with charges.

In [ ]:
def build_system(n, extent=(10, 10, 10), q=(-10, 10)):
    """Constucts a trial system in cartesian coordinates with random charges.
    """
    np.random.seed(seed=1) # If you want to get same results every time
    xyz = np.random.rand(n, 3) * np.array(extent)
    q = np.random.randint(*q, size=n)
    return (xyz, q)

In [ ]:
xyz, q = build_system(1000)
# print(xyz)
# print(q)

### Pure Python loop

This will be pretty slow. 

`Note on timing`
The timeit output will state a time per loop - a loop here refers to one run of the function (not the inner loop!). With small functions, timeit will do multiple iterations of the function to reduce the impact of timing overhead. It will also repeat the timing run several times and take a mean and standard deviation. These can be controlled with arguments if required.

In [ ]:
def naive_electrostatics(xyz, q):
    """Electostatics with simply python for loops
    """
    ret = 0.0
    size = xyz.shape[0]
    for x in range(size):
        for y in range(x + 1, size):
            dx = xyz[x, 0] - xyz[y, 0]
            dy = xyz[x, 1] - xyz[y, 1]
            dz = xyz[x, 2] - xyz[y, 2]
            
            r = (dx * dx + dy * dy + dz * dz) ** 0.5
            ret += q[x] * q[y] / r
    return ret

print(naive_electrostatics(xyz, q))
ttime = %timeit -o naive_electrostatics(xyz, q)
print(ttime.worst)

### Now using NumPy
NumPy provides C-compiled functions and contiguous arrays through a Python interface.

In [ ]:
def numpy_electrostatics(xyz, q):
    """Electostatics with simply python for loops
    """
    ret = 0.0
    size = xyz.shape[0]
    for x in range(size):
        dxyz = xyz[:x] - xyz[x]
        r = np.sqrt(np.einsum('ij,ij->i', dxyz, dxyz))
        ret += np.sum(q[:x] * q[x] / r)
    return ret

print(numpy_electrostatics(xyz, q))
%timeit numpy_electrostatics(xyz, q)

## Cython Implementations

### Unmodified code
This cell compiles with Cython, and shows the Cython generated code. It does NOT run it!  
This one is unmodified Python, so the code will be making CPython calls (Note the yellow lines in the output).

In [ ]:
%%cython --annotate

def cython1_electrostatics(xyz, q):
    """Electostatics with simply python for loops
    """
    ret = 0.0
    size = xyz.shape[0]
    for x in range(size):
        for y in range(x + 1, size):
            dx = xyz[x, 0] - xyz[y, 0]
            dy = xyz[x, 1] - xyz[y, 1]
            dz = xyz[x, 2] - xyz[y, 2]
            
            r = (dx * dx + dy * dy + dz * dz) ** 0.5
            ret += q[x] * q[y] / r
    return ret


**Still slow!**
It's essentially running Python

In [ ]:
print(cython1_electrostatics(xyz, q))
%timeit cython1_electrostatics(xyz, q)

### Cython with type definitions and NumPy arrays

In [ ]:
%%cython --annotate

cimport cython
cimport numpy as np

def cython2_electrostatics(np.ndarray[np.double_t, ndim=2] xyz, np.ndarray[np.int_t, ndim=1] q):
    """Electostatics with simply python for loops
    """
    cdef np.double_t ret = 0.0
    cdef int size = xyz.shape[0]
    
    cdef np.double_t dx, dy, dz, r
    cdef int x, y
    
    for x in range(size):
        for y in range(x + 1, size):
            dx = xyz[x, 0] - xyz[y, 0]
            dy = xyz[x, 1] - xyz[y, 1]
            dz = xyz[x, 2] - xyz[y, 2]
            
            r = (dx * dx + dy * dy + dz * dz) ** 0.5
            ret += q[x] * q[y] / r
    return ret

In [ ]:
print(cython2_electrostatics(xyz, q))
%timeit cython2_electrostatics(xyz, q)

### Parallel Cython

This should run using multiple threads and run faster than the serial version.
Use top to check if multiple threads are running.
Cython uses OpenMP for parallelism so the %%cython compile args will need to match your compiler. Note also, that in the prange line the GIL is released.

In [ ]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force

cimport cython
cimport cython.parallel
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True) 
def cython3_electrostatics(np.ndarray[np.double_t, ndim=2] xyz, np.ndarray[np.int_t, ndim=1] q):
    """Electostatics with simply python for loops
    """
    cdef np.double_t ret = 0.0
    cdef int size = xyz.shape[0]
    
    cdef np.double_t dx, dy, dz, r
    cdef int x, y
    
    for x in cython.parallel.prange(size, nogil=True, schedule="guided"):
        for y in range(x + 1, size):
            dx = xyz[x, 0] - xyz[y, 0]
            dy = xyz[x, 1] - xyz[y, 1]
            dz = xyz[x, 2] - xyz[y, 2]
            
            r = (dx * dx + dy * dy + dz * dz) ** 0.5
            ret += q[x] * q[y] / r
    return ret

In [ ]:
print(cython3_electrostatics(xyz, q))
%timeit cython3_electrostatics(xyz, q)

## Numba

Numba will jit compile the pure Python code, without requiring types.
It uses LLVM as the back end. When you install Numba (e.g. pip or conda), it will install llvmlite as a dependency, so you don't need to already have it on your system.

In [ ]:
import numba
jit_electrostatics = numba.jit(naive_electrostatics, nopython=True)

print(jit_electrostatics(xyz, q))
%timeit jit_electrostatics(xyz, q)

### Parallel Numba

In [ ]:
def prange_electrostatics(xyz, q):
    """Electostatics with simply python for loops
    """
    ret = 0.0
    size = xyz.shape[0]
    for x in numba.prange(size):
        for y in range(x + 1, size):
            dx = xyz[x, 0] - xyz[y, 0]
            dy = xyz[x, 1] - xyz[y, 1]
            dz = xyz[x, 2] - xyz[y, 2]
            
            r = (dx * dx + dy * dy + dz * dz) ** 0.5
            ret += q[x] * q[y] / r
    return ret

jit_prange_electrostatics = numba.jit(prange_electrostatics, nopython=True, parallel=True)

print(jit_prange_electrostatics(xyz, q))
%timeit jit_prange_electrostatics(xyz, q)

### Numba on GPU
Uncomment and try if you have a CUDA capable GPU

In [ ]:
# Required CUDA GPU's 'conda install cudatoolkit
#gpujit_prange_electrostatics = numba.cuda.jit(prange_electrostatics, nopython=True)

# print(gpujit_prange_electrostatics(xyz, q))
# %timeit gpujit_prange_electrostatics(xyz, q)

## Comparison of methods
Compare across implementations for various size datasets.

In [ ]:
import seaborn
import pandas as pd

func_dict = {
    "python": naive_electrostatics,
    "numpy": numpy_electrostatics,
    "cython2": cython2_electrostatics,
    "cython3": cython3_electrostatics,
    "jit": jit_electrostatics,
    "jit_prange": jit_prange_electrostatics,
}

active = {k : True for k in func_dict.keys()}

sizes = [1e1, 1e2, 1e3, 1e4]
df = []

for s in sizes:
    tmp_xyz, tmp_q = build_system(int(s))
    print("\nTimes for size {}".format(s))
    print("----------------------------")
    for name, func in func_dict.items():
        

        # Filter out very long runs
        if not active[name]:
            continue
            
        print("\nTiming function {}:".format(name))

        
        timing = %timeit -o func(tmp_xyz, tmp_q)
        df.append([s, name, timing.best])
        
        if timing.best > 0.5:
            active[name] = False

df = pd.DataFrame(df, columns=["Size", "Algorithm", "Time [s]"])

### Show table of results

In [ ]:
df

### Plot results

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(font_scale=2)
fig, ax = plt.subplots()
fig.set_size_inches(16, 12)

g = sns.barplot(x="Size", y="Time [s]", hue="Algorithm", data=df, ax=ax)
g.set_yscale('log')